In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML, display

# Cargar el conjunto de datos
file_path = '/Titanic-Dataset.csv'
titanic_data = pd.read_csv(file_path)

# Mostrar las primeras filas del conjunto de datos
titanic_data.head()

# Verificar valores faltantes y obtener un resumen del conjunto de datos
missing_values = titanic_data.isnull().sum()
summary = titanic_data.describe(include='all')

missing_values, summary

# Manejo de valores faltantes

# Imputar edades faltantes con la edad mediana
titanic_data['Age'].fillna(titanic_data['Age'].median(), inplace=True)

# Imputar valores faltantes de 'Embarked' con la moda (valor más común)
titanic_data['Embarked'].fillna(titanic_data['Embarked'].mode()[0], inplace=True)

# Eliminar la columna 'Cabin' debido a la gran cantidad de valores faltantes
titanic_data.drop(columns=['Cabin'], inplace=True)

# Ingeniería de características

# Convertir características categóricas ('Sex' y 'Embarked') en representaciones numéricas
titanic_data['Sex'] = titanic_data['Sex'].map({'male': 0, 'female': 1})
titanic_data = pd.get_dummies(titanic_data, columns=['Embarked'], drop_first=True)

# Crear una nueva característica 'FamilySize' combinando 'SibSp' y 'Parch'
titanic_data['FamilySize'] = titanic_data['SibSp'] + titanic_data['Parch']

# Eliminar columnas innecesarias
titanic_data.drop(columns=['Name', 'Ticket'], inplace=True)

# Mostrar las primeras filas del conjunto de datos limpio y con ingeniería de características
titanic_data.head()

# Definir características y objetivo
X = titanic_data.drop(columns=['Survived', 'PassengerId'])
y = titanic_data['Survived']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo de Regresión Logística
logreg = LogisticRegression(max_iter=200)
logreg.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = logreg.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Precisión:", accuracy)
print("Matriz de Confusión:\n", conf_matrix)
print("Informe de Clasificación:\n", class_report)

# Visualización de resultados

# Matriz de confusión
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Realidad')
plt.show()

# Gráficos de barras para la precisión, precisión, recall y f1-score
report = classification_report(y_test, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()

plt.figure(figsize=(10, 7))
sns.barplot(x=report_df.index[:-3], y='f1-score', data=report_df[:-3])
plt.title('F1-Score por Clase')
plt.xlabel('Clase')
plt.ylabel('F1-Score')
plt.show()

# Mostrar el informe de clasificación como una tabla HTML
display(HTML(report_df.to_html()))

# Mostrar precisión del modelo
print(f"Precisión del modelo: {accuracy:.2f}")